# SAÉ Labyrinthes



In [132]:
from random import randint, choice, shuffle

## 3. Implémentation
Définition de la classe `Maze`

In [186]:
class Maze:
    """
    Classe Labyrinthe
    Représentation sous forme de graphe non-orienté
    dont chaque sommet est une cellule (un tuple (l,c))
    et dont la structure est représentée par un dictionnaire
      - clés : sommets
      - valeurs : ensemble des sommets voisins accessibles
    """
    def __init__(self, height, width):
        """
        Constructeur d'un labyrinthe de height cellules de haut 
        et de width cellules de large 
        Les voisinages sont initialisés à des ensembles vides
        Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
        """
        self.height    = height
        self.width     = width
        self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}

    def info(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
        Retour:
            chaîne (string): description textuelle des attributs de l'objet
        """
        txt = "**Informations sur le labyrinthe**\n"
        txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
        txt += "- Voisinages :\n"
        txt += str(self.neighbors)+"\n"
        valid = True
        for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
            for c2 in self.neighbors[c1]:
                if c1 not in self.neighbors[c2]:
                    valid = False
                    break
            else:
                continue
            break
        txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
        return txt

    def __str__(self):
        """
        **NE PAS MODIFIER CETTE MÉTHODE**
        Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
        Retour:
             chaîne (str) : chaîne de caractères représentant le labyrinthe
        """
        txt = ""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
        txt += "   ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"

        return txt
    
    #--------------------------------------------------#
    
    def add_wall(self, c1:tuple, c2:tuple) -> None:
        """Méthode qui permet de créer un mur.
        Un mur est entre deux cellules.

        Args:
            c1 (tuple): Cellule d'un côté du mur.
            c2 (tuple): Cellule de l'autre côté du mur.
        
        Returns:
            None: Ne renvoie rien.
        """
        # Facultatif : on teste si les sommets sont bien dans le labyrinthe
        assert 0 <= c1[0] < self.height and \
            0 <= c1[1] < self.width and \
            0 <= c2[0] < self.height and \
            0 <= c2[1] < self.width, \
            f"Erreur lors de l'ajout d'un mur entre {c1} et {c2} : les coordonnées de sont pas compatibles avec les dimensions du labyrinthe"
        # Ajout du mur
        if c2 in self.neighbors[c1]:      # Si c2 est dans les voisines de c1
            self.neighbors[c1].remove(c2) # on le retire
        if c1 in self.neighbors[c2]:      # Si c3 est dans les voisines de c2
            self.neighbors[c2].remove(c1) # on le retire
        return None
            
    def remove_wall(self, c1:tuple, c2:tuple) -> None:
        """Méthode qui permet de supprimer un mur.
        Un mur est entre deux cellules.

        Args:
            c1 (tuple): Cellule d'un côté du mur.
            c2 (tuple): Cellule de l'autre côté du mur.
        
        Returns:
            None: Ne renvoie rien.
        """
        # Suppression du mur
        if c2 not in self.neighbors[c1]:      # Si c2 n'est pas dans les voisines de c1
            self.neighbors[c1].add(c2) # on l'ajoute
        if c1 not in self.neighbors[c2]:      # Si c1 n'est pas dans les voisines de c2
            self.neighbors[c2].add(c1) # on l'ajoute
        return None
    
    def get_cells(self) -> list:
        """Méthode qui renvoie une liste de toutes les cellules
        du labyrinthe.

        Returns:
            list: Listes des cellules du labyrinthe.
        """
        L = []
        for i in range(0,self.height):
            for j in range(0, self.width):
                L.append((i,j))
        return L
    
    def inGrid(self, c: tuple) -> bool:
        """Méthode permettant de tester si une cellule
        est dans la grille du labyrinthe ou non.

        Args:
            c (tuple): Cellule observée.

        Returns:
            bool: Renvoie True si elle est dans la grille, False sinon.
        """
        return (0 <= c[0] < self.height) and (0 <= c[1] < self.width)
    
    def get_walls(self) -> list:
        """Méthode qui renvoie une liste de tous les murs du labyrinthe.

        Returns:
            list: Liste de tuples des murs.
        """
        L = []
        lstCells = self.get_cells()
        for c1 in lstCells:
            # Création des voisins de c1, c2 pour l'Est et c3 pour le Sud.
            c2 = (c1[0], c1[1] + 1)
            c3 = (c1[0] + 1, c1[1])
            if self.inGrid(c2) and (c2 not in self.neighbors[c1]):
                L.append((c1,c2))
            if self.inGrid(c3) and (c3 not in self.neighbors[c1]):
                L.append((c1, c3))
        return L
    
    def fill(self) -> None:
        """Méthode qui modifie le labyrinthe et le rend
        complet, avec des murs partout.
        
        Returns:
            None: Ne renvoie rien.
        """
        lstCells = self.get_cells()
        for c1 in lstCells:
            # Création des voisins de c1, c2 pour l'Est et c3 pour le Sud.
            c2 = (c1[0], c1[1] + 1)
            c3 = (c1[0] + 1, c1[1])
            if self.inGrid(c2):
                self.add_wall(c1, c2)
            if self.inGrid(c3):
                self.add_wall(c1, c3)
        return None
    
    def empty(self) -> None:
        """Méthode qui modifie le labyrinthe et le rend
        vide, sans murs.
        
        Returns:
            None: Ne renvoie rien.
        """
        lstCells = self.get_cells()
        for c1 in lstCells:
            # Création des voisins de c1, c2 pour l'Est et c3 pour le Sud.
            c2 = (c1[0], c1[1] + 1)
            c3 = (c1[0] + 1, c1[1])
            if self.inGrid(c2):
                self.remove_wall(c1, c2)
            if self.inGrid(c3):
                self.remove_wall(c1, c3)
        return None
    
    def get_contiguous_cells(self, c:tuple) -> list:
        """Méthode qui renvoie les une liste de cellules contigues
        de 'c' dans la grille.

        Args:
            c (tuple): Cellule cible.

        Returns:
            list: Liste des cellules.
        """
        L = []
        cE = (c[0], c[1] + 1) # Cellule Est (Droite)
        cS = (c[0] + 1, c[1]) # Cellule Sud (Bas)
        cW = (c[0], c[1] - 1) # Cellule West (Gauche)
        cN = (c[0] - 1, c[1]) # Cellule North (Haut)
        # On teste si on peut les placer sur la grille
        if self.inGrid(cE):
            L.append(cE)
        if self.inGrid(cS):
            L.append(cS)
        if self.inGrid(cW):
            L.append(cW)
        if self.inGrid(cN):
            L.append(cN)
        return L
    
    def get_reachable_cells(self, c:tuple) -> list:
        """Méthode qui renvoie les une liste de cellules voisines
        contigues de 'c' dans la grille.

        Args:
            c (tuple): Cellule cible.

        Returns:
            list: Liste des cellules.
        """
        L = []
        cE = (c[0], c[1] + 1) # Cellule Est (Droite)
        cS = (c[0] + 1, c[1]) # Cellule Sud (Bas)
        cW = (c[0], c[1] - 1) # Cellule West (Gauche)
        cN = (c[0] - 1, c[1]) # Cellule North (Haut)
        # On teste si on peut les placer sur la grille
        if self.inGrid(cE) and (cE in self.neighbors[c]):
            L.append(cE)
        if self.inGrid(cS) and (cS in self.neighbors[c]):
            L.append(cS)
        if self.inGrid(cW) and (cW in self.neighbors[c]):
            L.append(cW)
        if self.inGrid(cN) and (cN in self.neighbors[c]):
            L.append(cN)
        return L
    
    #--------------------------------------------------#
    
    @classmethod
    def gen_btree(cls, h:int, w:int) -> object:
        """Méthode de classe qui créer un labyrinthe selon l'algorithme
        de l'arbre binaire.

        Args:
            h (int): heigtht, hauteur du labyrinthe créé.
            w (int): width, largeur du labyrinthe créé.

        Returns:
            object: Renvoie labyrinthe déjà créé.
        """
        laby = cls(h, w)
        for i in range(laby.height):
            for j in range(laby.width): # Optimisable avec get_cells() ?
                c = (i, j)
                cE = (c[0], c[1] + 1)
                cS = (c[0] + 1, c[1])
                lstC = [cE, cS]
                # Si cellule Est est dans les voisines condigues de 'c' et que cellule Sud
                # est dans les voisines contigues de 'c', alors on en supprime une des deux au hasard
                if cE in laby.get_contiguous_cells(c) and cS in laby.get_contiguous_cells(c): 
                    laby.remove_wall(c, lstC[randint(0,1)])
                # Si cellule Est est dans les voisines condigues de 'c' mais pas cellule Sud
                # Alors on supprime cellule Est
                elif cE in laby.get_contiguous_cells(c) and cS not in laby.get_contiguous_cells(c):
                    laby.remove_wall(c, cE)
                # Meme principe
                elif cE not in laby.get_contiguous_cells(c) and cS in laby.get_contiguous_cells(c):
                    laby.remove_wall(c, cS)
        return laby
    
    @classmethod
    def gen_sidewinder(cls,h,w):
        """Méthode de classe qui créer un labyrinthe selon l'algorithme
        Sidewinder.

        Args:
            h (int): heigtht, hauteur du labyrinthe créé.
            w (int): width, largeur du labyrinthe créé.

        Returns:
            object: Renvoie labyrinthe déjà créé.
        """
        laby=cls(w,h)
        laby.fill()
        for i in range (0,h-1):
            sequence = []
            for j in range(0,w-1):
                sequence+=[(i,j)]
                a = randint(1,2) #pile ou face
                if a == 1:
                    rand=randint(sequence[0][1],sequence[0][1]+len(sequence)-1) #nombre compris entrela colonne la plus basse d'une séquence au plus élevé
                    laby.remove_wall((i,rand),(i+1,rand))
                    sequence = []
                else :
                    laby.remove_wall((i,j),(i,j+1))
            sequence+=[(i,w-1)]
            rand=randint(sequence[0][1],sequence[0][1]+len(sequence)-1) #nombre compris entrela colonne la plus basse d'une séquence au plus élevé
            laby.remove_wall( (i,rand),(i+1,rand))
        for i in range(0,w-1):
            laby.remove_wall((h-1,i),(h-1,i+1))
        return laby
            
    @classmethod
    def gen_fusion(cls,h,w):
        """Méthode de classe qui créer un labyrinthe selon l'algorithme
        de fusion de chemins.

        Args:
            h (int): heigtht, hauteur du labyrinthe créé.
            w (int): width, largeur du labyrinthe créé.

        Returns:
            object: Renvoie labyrinthe déjà créé.
        """
        lab=cls(h,w)
        lab.fill()
        numCell=[]# on aura 0=(0,0) puis 1=(0,1) puis 3=(0,2) puis 3=(0,3) ... w-1=(0,w-1) puis w=(1,0) dc pr les cell de la ligne 2 par exemple elles seront entre 2*w et 2*w+h-1 compris
        for i in range(0,h):
            for j in range (0,w):
                numCell+=[i*h+j]
        walls=lab.get_walls()
        shuffle(walls)
        for wall in walls:#on s'intéresse aux mur un mur sépare deux cellules
            c1=wall[0] #voici la première
            c2=wall[1] #et la seconde
            #2 notions: le numéro de cellule: ce qui est calculé pour casser un mur ou pas et l'indice de cellule qui vaudra toujours le numéro de cellule du départ, (on peut le calculer)
            numC1=numCell[c1[0]*h+c1[1]] #on cherche son numéro de cellule grace à l'indice de cellule 
            numC2=numCell[c2[0]*h+c2[1]] #on cherche son numéro de cellule grace à l'indice de cellule 
            
            if numC1>numC2: # on veut que si les numéros de cellules sont différent, on :
                lab.remove_wall(c1,c2) # 1.casse le mur
                i=0
                while i<len(numCell) and numC1 in numCell:
                    if numCell[i]==numC1: #2. affecter le numéro de cellule le moins élevés aux cellules ayant le plus élevé
                        numCell[i]=numC2
                    i+=1
                    
            elif numC1<numC2:
                lab.remove_wall(c1,c2)
                i=0
                while i<len(numCell) and numC2 in numCell:
                    if numCell[i]==numC2:
                        numCell[i]=numC1
                    i+=1
        return lab

    @classmethod
    def gen_exploration(cls,h,w):
        """Méthode de classe qui créer un labyrinthe selon l'algorithme
        de l'Exploration exhaustive.

        Args:
            h (int): heigtht, hauteur du labyrinthe créé.
            w (int): width, largeur du labyrinthe créé.

        Returns:
            object: Renvoie labyrinthe déjà créé.
        """
        laby = cls(h,w)
        c = (randint(0,laby.height-1),randint(0,laby.width-1))
        pile = [c]
        marque=[] # Crém ed engil enu'd noit
        for i in range (0,laby.height): 
            for _ in range(0,laby.width):
                marque+=[False]
        marque[c[0]*(laby.height)+c[1]] = True # Création d'un identifiant unique qui ne sortira pas de la liste.
        while len(pile) > 0:
            c=pile.pop()
            cE = (c[0], c[1] + 1)
            cS = (c[0] + 1, c[1])
            cN = (c[0]-1,c[1])
            cO = (c[0],c[1] - 1)
            vois=[cE,cS,cN,cO]
            voisNM=[]
            for i in range(0,4):
                if vois[i][0]*(laby.height)+vois[i][1]<laby.height*laby.height and marque[vois[i][0]*(laby.height)+vois[i][1]] == False and laby.inGrid(vois[i]):
                    voisNM+=[vois[i]]
            if voisNM!=[]:
                pile+=[c]
                voisAlea=voisNM[randint(0,len(voisNM)-1)] 
                laby.remove_wall(c,voisAlea)
                marque[voisAlea[0]*(laby.height)+voisAlea[1]]=True
                pile+=[voisAlea]
        return laby
    
    @classmethod
    def gen_wilson(cls, h: int, w:int) -> object:
        """Méthode de classe qui permet de créer un labyrinthe
        selon l'algorithme de Wilson.

        Args:
            h (int): heigtht, hauteur du labyrinthe créé.
            w (int): width, largeur du labyrinthe créé. 

        Returns:
            object: Renvoie le labyrinthe déjà créé.
        """
        laby = cls(h, w)
        # On choisi une cellule au hasard sur la grille
        cells = laby.get_cells()
        c = choice(cells)
        # Création d'un dictionnaire de cellules non marqués, on marque notre cellule de départ
        marked = [c]
        # Tant qu'il reste des cellules non marqués
        while len(marked) < len(cells):
            
            # On choisi une cellule non marqué aléatoire dans le dictionnaire
            c = choice(cells)
            while c in marked:
                c = choice(cells)
            
            # On effectue la marche aléatoire jusqu'à ce qu'une cellule déjà marqué soit atteinte.
            # On va marquer les cellules a la fin du parcours, pas au fur et à mesure.
            # Recherche des cellules voisines
            cPro = choice(cells) # Cellule prochaine
            while cPro in marked:
                cPro = choice(cells)
            chemin = [cPro]
            # Tant que la cellule n'est pas marqué
            while cPro not in marked:
                cPro = choice(laby.get_contiguous_cells(cPro))
                # Si on se mord la queue
                if cPro in chemin:
                     # On supprime la dernière cellule
                    del chemin[-1]
                    # On supprime toutes les cellules jusqu'à la cellule marqué
                    while chemin[-1] != cPro: 
                        del chemin[-1]
                # Sinon on continue
                else:
                    chemin.append(cPro)
            
            # On marque toutes les cellules et on casse les murs
            for l in range(len(chemin)-1):
                marked.append(chemin[l]) # On marque la cellule
                laby.remove_wall(chemin[l], chemin[l+1]) # On casse le mur
        return laby
    
    #--------------------------------------------------#
    
    def overlay(self, content=None):
        """
        Rendu en mode texte, sur la sortie standard, \
        d'un labyrinthe avec du contenu dans les cellules
        Argument:
            content (dict) : dictionnaire tq content[cell] contient le caractère à afficher au milieu de la cellule
        Retour:
            string
        """
        if content is None:
            content = {(i,j):' ' for i in range(self.height) for j in range(self.width)}
        else:
            # Python >=3.9
            #content = content | {(i, j): ' ' for i in range(
            #    self.height) for j in range(self.width) if (i,j) not in content}
            # Python <3.9
            new_content = {(i, j): ' ' for i in range(self.height) for j in range(self.width) if (i,j) not in content}
            content = {**content, **new_content}
        txt = r""
        # Première ligne
        txt += "┏"
        for j in range(self.width-1):
            txt += "━━━┳"
        txt += "━━━┓\n"
        txt += "┃"
        for j in range(self.width-1):
            txt += " "+content[(0,j)]+" ┃" if (0,j+1) not in self.neighbors[(0,j)] else " "+content[(0,j)]+"  "
        txt += " "+content[(0,self.width-1)]+" ┃\n"
        # Lignes normales
        for i in range(self.height-1):
            txt += "┣"
            for j in range(self.width-1):
                txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
            txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
            txt += "┃"
            for j in range(self.width):
                txt += " "+content[(i+1,j)]+" ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else " "+content[(i+1,j)]+"  "
            txt += "\n"
        # Bas du tableau
        txt += "┗"
        for i in range(self.width-1):
            txt += "━━━┻"
        txt += "━━━┛\n"
        return txt
    
    #--------------------------------------------------#
    
    def predecesseur(self, c:tuple) -> list:
        """Méthode qui renvoie la liste des prédécesseurs de la cellule 'c'.

        Args:
            c (tuple): Cellule cible.

        Returns:
            list: Liste des prédécesseurs de 'c'.
        """
        lst = []
        for i in range(self.height):
            for j in range(self.width):
                if c in self.neighbors[(i,j)]:
                    lst.append((i,j))
        return lst
        
    def solve_dfs(self, start: tuple, stop: tuple) -> list:
        """Méthode de résolution de parcours de labyrinthe en profondeur (pile).

        Args:
            start (tuple): Cellule de départ
            stop (tuple): Cellule d'arrivée
        Returns:
            list: Liste des cellules parcourues.
        """
        # Parcours du graphe jusqu’à ce qu’on trouve A        
        # Initialisation
        pile = [start]
        marked = [start]
        pred = {start : start}
        
        # Tant qu'il reste des cellules non marqués
        trouvé = False
        while not trouvé and len(marked) < len(self.get_cells()):
            c = pile.pop(-1)
            if c == stop:
                trouvé = True
            else:
                voisinC = self.get_reachable_cells(c)
                for voisin in voisinC:
                    if voisin not in marked:
                        marked.append(voisin)
                        pile.append(voisin)
                        pred[voisin] = c
        
        # Reconstruction du chemin à partir des prédécesseurs
        c = stop
        chemin = []
        while c != start:
            chemin.append(c)
            c = pred[c]
        chemin.append(start)
        
        return chemin
    
    def solve_bfs(self, start: tuple, stop: tuple) -> list:
        """Méthode de résolution de parcours de labyrinthe en largeur (file).

        Args:
            start (tuple): Cellule de départ
            stop (tuple): Cellule d'arrivée
        Returns:
            list: Liste des cellules parcourues.
        """
        # Parcours du graphe jusqu’à ce qu’on trouve A        
        # Initialisation
        pile = [start]
        marked = [start]
        pred = {start : start}
        
        # Tant qu'il reste des cellules non marqués
        trouvé = False
        while not trouvé and len(marked) < len(self.get_cells()):
            c = pile.pop(0)
            if c == stop:
                trouvé = True
            else:
                voisinC = self.get_reachable_cells(c)
                for voisin in voisinC:
                    if voisin not in marked:
                        marked.append(voisin)
                        pile.append(voisin)
                        pred[voisin] = c
        
        # Reconstruction du chemin à partir des prédécesseurs
        c = stop
        chemin = []
        while c != start:
            chemin.append(c)
            c = pred[c]
        chemin.append(start)
        
        return chemin
                  
    
    def card(self,c):
        return [(c[0],c[1]-1),(c[0]+1,c[1]),(c[0],c[1]+1),(c[0]-1,c[1])]#O,S,E,N
    
    def firstDepl(self,c):
        vois=self.card(c)
        cLibre=self.get_reachable_cells(c)
        if len(cLibre) == 1 :
            direction=vois.index(cLibre[0])
        elif len(cLibre) == 2:
            direction = vois.index(cLibre[randint(0,1)])
        else:
            direction = vois.index((cLibre[randint(0,1)]))+1%4       
        return vois[direction],(direction+2)%4
    
    def Deplacement(self,c,direction:int):#renvoi la direction et la case/hors 1er déplacement
        #direction = 0 pour Ouest, 1 pr sud
        #direction: je viens de/du ...
        #dir = la ou je vais
        dir=-1
        vois=self.card(c)
        cLibre=self.get_reachable_cells(c)
        if len(cLibre)==1:#demi tour
            dir=direction 
        elif len(cLibre)==2:
            if cLibre[0][0]==cLibre[1][0] or cLibre[0][1]==cLibre[1][1]:#cas d'un "tube", avec la cardinalité si on fait +2%2 on tombe sur direc oppo
                dir=(direction+2)%4
            else : #possible de condenser avec les modulo, rel math entre directions !avec un boucle!
                if vois[0] in cLibre and vois[1] in cLibre:#sud et ouest de dispo
                    dir = 0 if direction == 1 else 1
                elif vois[1] in cLibre and vois[2] in cLibre: #sud et est de dispo
                    dir = 1 if direction == 2 else 2
                elif vois[0] in cLibre and vois[3] in cLibre: #ouest et nord de disponible
                    dir = 0 if direction == 3 else 3
                else: #cellule nord et best de dispo
                    dir = 2  if direction == 3 else 3
        elif len(cLibre)==4:
            dir=(direction+1)%4
        else :#cas dur qui prend en compte dir
            #definir que i c'est la case de direction+1
            dir = (direction+1)%4
            while vois[(dir%4)] not in cLibre :
                dir= (dir+1)%4
        return vois[dir],(dir+2)%4 #si je vais à l'est je viens de l'ouest 
    
    def solve_hrh(self,start,stop):
        #premier déplacement (on coco pas direction)
        c,direction = self.firstDepl(start)
        pile = [c]
        while c != stop:
            c,direction=self.Deplacement(c,direction)
            if c not in pile:
                pile+=[c]
        return pile
    
    def distance_geo(self, c1: tuple, c2: tuple) -> int:
        """Méthode qui donne la distance géodésique de deux points sur le labyrinthe.

        Args:
            c1 (tuple): Cellule de départ
            c2 (tuple): Cellule d'arrivée

        Returns:
            int: Distance entre les deux points
        """
        return len(self.solve_bfs(c1, c2)) -1
        

## 4. Manipulation de labyrinthes

In [134]:
laby = Maze(5, 5)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



Ajout de la méthode `remove_wall(c1, c2)` dans la classe `Maze`

In [135]:
laby.remove_wall((0,0),(0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛




Ajout de la méthode `add_wall(c1, c2)` dans la classe `Maze`

In [136]:
laby.add_wall((0,0),(0,1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



Ajout de la méthode `fill()` dans la classe `Maze`

In [137]:
laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



Ajout de la méthode `empty()` dans la classe `Maze`

In [138]:
laby.empty()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [139]:
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



Ajout de la méthode `get_walls()` dans la classe `Maze`

In [140]:
print(laby.get_walls())

[((0, 0), (0, 1)), ((0, 1), (1, 1))]


Ajout de la méthode `get_contiguous_cells(c)` dans la classe `Maze`

In [141]:
print(laby.get_contiguous_cells((0,1)))

[(0, 2), (1, 1), (0, 0)]


Ajout de la méthode `get_reachable_cells(c)` dans la classe `Maze`

In [142]:
print(laby.get_reachable_cells((0,1)))

[(0, 2)]


Ajout de la fonction `get_cells()` dans la classe `Maze`

In [143]:
print(laby.get_cells())

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]


## 5. Génération

Nous aborderons les algorithmes de créations de labyrinthes parfaits avec les algo : 
- Par arbre binaire
- De Sidewinder
- Par fusion de chemins
- Par exploration exhaustive
- De Wilson

### 5.1 Arbre binaire

In [144]:
laby = Maze.gen_btree(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃       ┃       ┃
┣━━━╋   ╋━━━╋   ┫
┃       ┃       ┃
┣━━━╋   ╋━━━╋   ┫
┃               ┃
┣━━━╋━━━╋━━━╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



### 5.2 Sidewinder

In [145]:
laby = Maze.gen_sidewinder(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃           ┃   ┃
┣━━━╋   ╋━━━╋   ┫
┃   ┃   ┃       ┃
┣   ╋   ╋   ╋━━━┫
┃           ┃   ┃
┣━━━╋━━━╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



### 5.3 Fusion de chemins

In [146]:
laby = Maze.gen_fusion(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃                   ┃   ┃       ┃       ┃   ┃       ┃
┣━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━┫
┃       ┃           ┃   ┃   ┃                               ┃
┣   ╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━┫
┃   ┃           ┃   ┃   ┃   ┃   ┃   ┃       ┃   ┃   ┃       ┃
┣   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ┫
┃   ┃   ┃       ┃   ┃           ┃       ┃   ┃   ┃           ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━┫
┃       ┃           ┃               ┃   ┃   ┃   ┃       ┃   ┃
┣━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃               ┃                   ┃   ┃           ┃       ┃
┣   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃           ┃   ┃   ┃   ┃       ┃       ┃           ┃       ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋   ┫
┃       ┃       ┃   ┃   ┃   ┃   ┃   ┃       ┃   ┃           ┃
┣   ╋   

### 5.4 Exploration exhaustive

In [147]:
laby = Maze.gen_exploration(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃               ┃                                   ┃       ┃
┣━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━┫
┃       ┃   ┃   ┃           ┃                       ┃       ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃       ┃       ┃   ┃       ┃       ┃                   ┃   ┃
┣━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ┫
┃       ┃       ┃       ┃       ┃   ┃   ┃           ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ┫
┃           ┃           ┃       ┃           ┃       ┃       ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━┫
┃           ┃       ┃   ┃                       ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ┫
┃   ┃           ┃   ┃           ┃           ┃           ┃   ┃
┣   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃   ┃   ┃       ┃       ┃       ┃   ┃       ┃   ┃           ┃
┣   ╋   

### 5.5 Wilson

In [148]:
laby = Maze.gen_wilson(12, 12)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃               ┃                   ┃       ┃   ┃
┣━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃       ┃   ┃               ┃   ┃       ┃       ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━┫
┃       ┃       ┃   ┃   ┃           ┃   ┃   ┃   ┃
┣━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ┫
┃           ┃           ┃   ┃       ┃   ┃       ┃
┣━━━╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ┫
┃           ┃   ┃   ┃   ┃       ┃   ┃       ┃   ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ┫
┃   ┃   ┃       ┃   ┃       ┃   ┃   ┃       ┃   ┃
┣━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋   ╋   ┫
┃   ┃   ┃           ┃   ┃   ┃       ┃   ┃       ┃
┣   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━┫
┃       ┃       ┃   ┃       ┃       ┃           ┃
┣   ╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━┫
┃   ┃       ┃   ┃               ┃       ┃       ┃
┣   ╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃   ┃           ┃   ┃       ┃           ┃   ┃


## 6. Résolution

Ajout de la méthode `overlay` dans la classe `Maze`

### 6.1 Résolution par parcours

In [184]:
laby = Maze.gen_wilson(13,13)
solution = laby.solve_dfs((0, 0), (12, 12))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(12, 12)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *     ┃                   ┃   ┃               ┃
┣━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ┫
┃     *     ┃   ┃       ┃   ┃       ┃       ┃   ┃   ┃
┣━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ┫
┃     * ┃ *   *   * ┃       ┃   ┃           ┃       ┃
┣━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━┫
┃     *   *     ┃ *     ┃   ┃       ┃               ┃
┣   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ┫
┃           ┃     *     ┃       ┃       ┃   ┃       ┃
┣   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━┫
┃   ┃   ┃       ┃ *     ┃   ┃       ┃   ┃       ┃   ┃
┣   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃       ┃       ┃ *   *         ┃       ┃           ┃
┣   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃   ┃   ┃   ┃         * ┃           ┃   ┃       ┃   ┃
┣━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━┫
┃       ┃   ┃   ┃ *   * ┃   ┃       ┃   ┃           ┃
┣   ╋━━━╋   ╋   ╋   ╋━━━╋   

In [185]:
laby = Maze.gen_wilson(13,13)
solution = laby.solve_bfs((0, 0), (12,12))
print(laby.distance_geo((0, 0), (12,12)))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(12, 12)] = 'A'
print(laby.overlay(str_solution))

35
┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D ┃               ┃               ┃   ┃       ┃   ┃
┣   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ┫
┃ *   *   * ┃   ┃   ┃   ┃   ┃                       ┃
┣━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃   ┃   ┃ * ┃       ┃   ┃   ┃       ┃   ┃   ┃   ┃   ┃
┣   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━┫
┃ *   *   * ┃   ┃           ┃   ┃       ┃   ┃       ┃
┣   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ┫
┃ *     ┃       ┃   ┃   ┃   ┃       ┃           ┃   ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ┫
┃ *         ┃                           ┃       ┃   ┃
┣   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ┫
┃ *     ┃           ┃   ┃           ┃   ┃   ┃   ┃   ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━┫
┃ *   *     ┃   ┃   ┃                   ┃   ┃       ┃
┣━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━┫
┃ *   * ┃                   ┃       ┃               ┃
┣   ╋   ╋━━━╋━━━╋   ╋━━━╋

## 6.2 Résolution en aveugle : « la main droite »

In [ ]:
laby = Maze.gen_wilson(13,13)
solution = laby.solve_hrh((0, 0), (12,12))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(4, 4)] = 'A'
print("--------------")
print(laby.overlay(str_solution))

--------------
┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   * ┃   ┃           ┃   ┃   ┃       ┃           ┃
┣   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━┫
┃ * ┃ *   *   *   *   *   *     ┃           ┃   ┃   ┃
┣   ╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ┫
┃ * ┃ * ┃ * ┃ *   *   *   *             ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━┫
┃ *   * ┃ * ┃ * ┃       ┃   ┃   ┃           ┃       ┃
┣   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━┫
┃ * ┃ *   * ┃ *   A   * ┃   ┃   ┃       ┃           ┃
┣━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ┫
┃ * ┃ *   * ┃ * ┃ * ┃ *   * ┃               ┃   ┃   ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃ * ┃ * ┃ * ┃ *   *   *   * ┃   ┃   ┃   ┃       ┃   ┃
┣   ╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃ *   *   * ┃ *     ┃   ┃   ┃       ┃   ┃   ┃   ┃   ┃
┣━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━┫
┃ *   *   *   *   *   * ┃       ┃           ┃       ┃
┣   ╋━━━╋━━━╋